#ממשק האתר שלי

In [ ]:
!pip install Flask
!pip install pyngrok
!pip install joblib
!pip install pandas
!pip install numpy
!pip install Flask pyngrok joblib
!pip install Flask flask-ngrok joblib pandas numpy qrcode[pil]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.9 MB/s eta 0:00:00


In [ ]:

from pyngrok import ngrok
os.makedirs("templates", exist_ok=True)
ngrok.set_auth_token("2v0a9ia4ml7K2VFmRpaY8UJ9kJ2_7zXk9LvhX2jdRFLgrV9q2")  # Replace with your token
# 4. Create necessary folders
!mkdir -p templates
!mkdir -p models

In [ ]:
from flask import Flask, request, send_file
from sklearn.preprocessing import LabelEncoder
import qrcode
from PIL import Image
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
import joblib
import pandas as pd
import os


# הגדרת ה-Flask app
app = Flask(__name__)

# טען את המודל שלך
model = joblib.load("/content/movie_success_model.joblib")
le_stars = joblib.load('/content/le_Stars.pkl')
le_languages = joblib.load('/content/le_spoken_languages.pkl')
le_genres = joblib.load('/content/le_genres.pkl')
le_directors = joblib.load('/content/le_Directors.pkl')
le_writers = joblib.load('/content/le_Writers.pkl')
# טען את הדאטה של הסרטים
movie_data = pd.read_csv('/content/adi_halel1.csv')
public_url = ngrok.connect(5000)
print(f"Flask app is running at: {public_url}")
#שילוב של QR
qr = qrcode.make(str(public_url))
qr.save("qr_code.png")
Image.open("qr_code.png").show()

if not os.path.exists("qr_code.png"):
    public_url = "http://localhost:5000"
    qr = qrcode.make(public_url)
    qr.save("qr_code.png")

@app.route('/static/qr_code.png')
def serve_qr_code():
    return send_file("qr_code.png", mimetype='image/png')

@app.route('/')
def home():
    return '''
    <!DOCTYPE html>
    <html>
        <head><title>QR Code</title></head>
            <body>
                <h1>Scan to site QR:</h1>
                <img src="/static/qr_code.png" alt="QR Code">
            </body>
        <head>
            <title>Movie Success Predictor</title>
            <style>
                body {
                    font-family: 'Arial', sans-serif;
                    background-color: #000000;
                    color: #fff;
                    text-align: center;
                    padding: 50px;
                    background-image: url('https://image.shutterstock.com/image-photo/cinema-movie-clapper-board-cinema-abstract-260nw-1488269386.jpg');
                    background-size: cover;
                    background-position: center;
                    background-repeat: no-repeat;
                    height: 100vh;
                    overflow-y: scroll;
                }
                form {
                    background-color: rgba(0, 0, 0, 0.75);
                    padding: 30px;
                    border-radius: 15px;
                    display: inline-block;
                    margin-top: 20px;
                    border: 3px solid #ff4757;
                    max-width: 400px;
                }
                input, button, select {
                    padding: 10px;
                    margin: 10px 0;
                    width: 100%;
                    border: none;
                    border-radius: 5px;
                    font-size: 16px;
                }
                button {
                    background-color: #ff4757;
                    color: white;
                    cursor: pointer;
                }
                button:hover {
                    background-color: #e84118;
                }
                h1 {
                    font-size: 42px;
                    margin-bottom: 10px;
                    text-shadow: 2px 2px 4px #000;
                }
                small {
                    display: block;
                    color: #ccc;
                    font-size: 12px;
                    margin-top: -8px;
                    margin-bottom: 10px;
                    text-align: left;
                }
            </style>
        </head>
        <body>
            <h1>🍿 Predict Movie Success 🎬</h1>

            <form action="/search" method="get" style="margin-bottom: 40px;">
                <input type="text" name="query_title" placeholder="🔍 Search movie by title..." required><br>
                <button type="submit">Search</button>
            </form>

            <form action="/predict" method="post">
                <input type="text" name="title" placeholder="Movie Title" title="Enter the movie's title" required><br>
                <small>Enter the movie's title (.e.g: Inception, Titanic) </small>

                <input type="number" name="runtime" placeholder="Runtime (in minutes)" title="Enter the movie's length in minutes (0-250)" min="0" max="250" required><br>
                <small>Length of the movie in minutes (.e.g: 120,205)</small>

                <select name="original_language" title="Original language of the movie" required>
                    <option value="1">English</option>
                    <option value="0">Other</option>
                </select><br>
                <small>If it's not English, mark it Other.</small>

                <input type="text" name="genres" placeholder="Genres" title="Enter one or more genres separated by commas" required><br>
                <small>Movie genres separated by commas (.e.g: Action, Drama..)</small>

                <input type="text" name="spoken_languages" placeholder="Spoken Languages" title="Languages spoken or dubbed in the movie" required><br>
                <small>Languages spoken in the movie separated by commas (.e.g: English, Arabic..)</small>

                <input type="number" name="year" placeholder="Year" title="Year the movie was released" required><br>
                <small>The year the movie was released (.e.g: 2023,2009,1990)</small>

                <select name="dayofweek" title="Day of the week the movie was released" required>
                      <option value="1">Sunday</option>
                      <option value="2">Monday</option>
                      <option value="3">Tuesday</option>
                      <option value="4">Wednesday</option>
                      <option value="5">Thursday</option>
                      <option value="6">Friday</option>
                      <option value="7">Saturday</option>
                                            </select><br>


                <input type="text" name="Directors" placeholder="Directors" title="List of movie directors" required><br>
                <small>List of directors (.e.g: Christopher Nolan,Lewis Allen)</small>

                <input type="text" name="Stars" placeholder="Stars" title="Names of the main actors" required><br>
                <small>Names of main actors separated by commas  (.e.g: Tom Hanks, Emma Stone)</small>

                <input type="text" name="Writers" placeholder="Writers" title="Names of the script writers" required><br>
                <small>Names of screenwriters separated by commas  (.e.g: Aaron Sorkin,Rama Burshtein)</small>

                <input type="number" name="Rating_Count" placeholder="Rating Count" title="Number of ratings the movie has received" required><br>
                <small>Total number of ratings (.e.g: 15000,90)</small>

                <input type="number" step="0.01" name="popularity" placeholder="Popularity" title="Popularity score (0–1000)" min="0" max="1000" required><br>
                <small>A decimal number representing the movie's popularity (.e.g: 582.34,80.0)</small>

                <input type="text" name="overview" placeholder="Overview (summary)" title="Short summary of the movie" required><br>
                <small>A brief summary of the movie (.e.g: 'A thief who...')</small>

                <input type="text" name="plot" placeholder="Plot (detailed)" title="Detailed storyline of the movie" required><br>
                <small>A more detailed description of the storyline</small>

                <button type="submit">Predict</button>
            </form>
    </body>
    </html>
    '''
le_genres.fit(['Action', 'Comedy', 'Drama', 'UNK'])
le_languages.fit(['English', 'French', 'Spanish', 'UNK'])
le_directors.fit(['Director A', 'Director B', 'UNK'])
le_stars.fit(['Star X', 'Star Y', 'UNK'])
le_writers.fit(['Writer M', 'Writer N', 'UNK'])
def safe_transform(encoder, values, default_value='UNK'):
    transformed_values = []
    for value in values:
        if value in encoder.classes_:
            transformed_values.append(encoder.transform([value])[0])
        else:
            # אם הערך לא נמצא, נשתמש בקטגוריה ברירת מחדל
            transformed_values.append(encoder.transform([default_value])[0])
    return transformed_values

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if request.method == 'POST':
            form = request.form
            print(f"Form data received: {form}")

            # קבלת הנתונים מהטופס
            title = form['title']
            runtime = form['runtime']
            original_language = form['original_language']
            genres = [g.strip() for g in form['genres'].split(',')]
            spoken_languages = [l.strip() for l in form['spoken_languages'].split(',')]
            year = form['year']
            dayofweek = form['dayofweek']

            Directors = [d.strip() for d in form['Directors'].split(',')]
            Stars = [s.strip() for s in form['Stars'].split(',')]
            Writers = [w.strip() for w in form['Writers'].split(',')]

            Rating_Count = form['Rating_Count']
            popularity = form['popularity']
            overview = form['overview']
            plot = form['plot']
            # בדיקת אם השדות מכילים ערכים תקינים
            if not title or not isinstance(title, str):
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Title' field must be text</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not runtime.isdigit() or float(runtime) <= 0:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Runtime' field must be a positive number</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if original_language not in ['0', '1']:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Original Language' field must be 'English' (0) or 'Other' (1)</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not isinstance(genres, list) or not genres:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Genres' field must be text</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not isinstance(spoken_languages, list) or not spoken_languages:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Spoken Languages' field must be text</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not year.isdigit():
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Year' field must be a number </h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if dayofweek not in ['1', '2', '3', '4', '5', '6', '7']:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Day of the Week' field must be between 1 and 7</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not isinstance(Directors, list) or not Directors:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Directors' field must be text</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not isinstance(Stars, list) or not Stars:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Stars' field must be text</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not isinstance(Writers, list) or not Writers:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Writers' field must be text</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not Rating_Count.isdigit() or float(Rating_Count) <= 0:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Rating Count' field must be a positive number</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """
            if not popularity.replace('.', '', 1).isdigit() or float(popularity) <= 0:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Popularity' field must be a positive number</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not isinstance(overview, str) or len(overview) == 0:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Overview' field must be text</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            if not isinstance(plot, str) or len(plot) == 0:
                return """
                <div style='background-color:black; color:white; padding:20px; text-align:center;'>
                    <h2>The 'Plot' field must be text</h2>
                    <p><a href='/' style='color: lightblue;'>Click here to return to the homepage</a></p>
                </div>
                """

            # המרת הרשימות באמצעות safe_transform
            genres = safe_transform(le_genres, genres)
            spoken_languages = safe_transform(le_languages, spoken_languages)
            Directors = safe_transform(le_directors, Directors)
            Stars = safe_transform(le_stars, Stars)
            Writers = safe_transform(le_writers, Writers)

            # המרת שדות נוספים
            runtime = float(runtime)
            original_language = int(original_language)
            year = int(year)
            dayofweek = int(dayofweek)
            Rating_Count = float(Rating_Count)
            popularity = float(popularity)

            overview_score = len(overview) % 10 / 10
            plot_score = len(plot) % 10 / 10

            # יצירת DataFrame עם הנתונים הממולאים
            data = pd.DataFrame([[year, genres[0], runtime, Stars[0], Rating_Count, popularity,
                                  Directors[0], spoken_languages[0], Writers[0], original_language,
                                  0.5, dayofweek, overview_score, plot_score]],
                                 columns=['year', 'genres', 'runtime', 'Stars', 'normalized_Rating_Count',
                                          'normalized_popularity', 'Directors', 'spoken_languages', 'Writers',
                                          'original_language', 'title_tfidf_max', 'dayofweek', 'overview_tfidf_max',
                                          'plot_tfidf_max'])

            # חיזוי באמצעות המודל
            prediction = model.predict(data)[0]

            # המרת המספר לתווית טקסט
            label_map = {
                1: "Loss",
                2: "Risk",
                3: "Successful",
                4: "Very Successful"
            }
            color_map = {
                "Loss": "red",
                "Risk": "orange",
                "Successful": "limegreen",
                "Very Successful": "gold"
            }

            label = label_map.get(prediction, "Unknown")
            color = color_map.get(label, "white")

            return f'''
                <html>
                    <head><title>Prediction Result</title></head>
                    <body style="text-align: center; font-family: Arial; background-color: #1c1c1c; color: white; padding: 100px;">
                        <h1>🎬 {title}</h1>
                        <h2>Predicted Success: <span style="color: {color};">{label}</span></h2>
                        <a href="/" style="color: #ff4757; text-decoration: none;">🔙 Try another movie</a>
                    </body>
                </html>
            '''

    except Exception as e:
        return f"Error: {str(e)}"


@app.route('/search')
def search():
    query = request.args.get('query_title', '').strip().lower()

    if not query:
        return """
        <html>
            <head>
                <title>Search Error</title>
                <style>
                    body {
                        background-color: #1c1c1c;
                        color: white;
                        font-family: Arial, sans-serif;
                        text-align: center;
                        padding: 50px;
                    }
                    input, button {
                        padding: 10px;
                        font-size: 16px;
                        border-radius: 5px;
                        border: none;
                        margin-top: 10px;
                    }
                    button {
                        background-color: #ff4757;
                        color: white;
                        cursor: pointer;
                    }
                    small {
                        color: #ccc;
                        font-size: 14px;
                        display: block;
                        margin-top: 8px;
                    }
                    a {
                        color: #ff4757;
                        display: block;
                        margin-top: 20px;
                    }
                </style>
            </head>
            <body>
                <h2>Please enter a movie title to search.</h2>
                <form action="/search" method="get">
                    <input type="text" name="query_title" placeholder="🔍 Search movie by title..." required><br>
                    <small>Enter the movie's title (e.g., Inception, Titanic)</small>
                    <button type="submit">Search</button>
                </form>
                <a href="/">🔙 Back to Home</a>
            </body>
        </html>
        """

    results = movie_data[
        (movie_data['title'].str.lower().str.contains(query)) &
        (movie_data['year'].between(1905, 2024))
    ]

    if results.empty:
        return f'''
        <html>
            <head>
                <title>Search Results</title>
                <style>
                    body {{
                        background-color: #1c1c1c;
                        color: white;
                        font-family: Arial, sans-serif;
                        text-align: center;
                        padding: 50px;
                    }}
                    input, button {{
                        padding: 10px;
                        font-size: 16px;
                        border-radius: 5px;
                        border: none;
                        margin-top: 10px;
                    }}
                    button {{
                        background-color: #ff4757;
                        color: white;
                        cursor: pointer;
                    }}
                    small {{
                        color: #ccc;
                        font-size: 14px;
                        display: block;
                        margin-top: 8px;
                    }}
                    a {{
                        color: #ff4757;
                        display: block;
                        margin-top: 20px;
                    }}
                </style>
            </head>
            <body>
                <h2>No results found for '{query}'</h2>
                <form action="/search" method="get">
                    <input type="text" name="query_title" placeholder="🔍 Search movie by title..." required><br>
                    <small>Enter the movie's title (e.g., Inception, Titanic)</small>
                    <button type="submit">Search</button>
                </form>
                <a href="/">🔙 Back to Home</a>
            </body>
        </html>
        '''

    label_map = {
        1: "Loss",
        2: "Risk",
        3: "Successful",
        4: "Very Successful"
    }

    result_html = "<h2 style='color:white; text-align:center;'>🎬 Search Results:</h2><ul style='color:white; text-align:left; width:60%; margin:auto;'>"
    for _, row in results.iterrows():
        movie_title = row['title']
        movie_year = row['year']
        movie_target = label_map.get(row['target'], 'Unknown')
        result_html += f"<li><strong>{movie_title}</strong> ({movie_year}) - <em>{movie_target}</em></li>"
    result_html += "</ul><div style='text-align:center; margin-top:20px;'><a href='/' style='color:#ff4757;'>🔙 Back to Home</a></div>"

    return f'''
        <html>
            <head><title>Search Results</title></head>
            <body style="background-color:#1c1c1c; font-family:Arial; color:white;">
                <form action="/search" method="get" style="margin-bottom: 40px; text-align:center;">
                    <input type="text" name="query_title" placeholder="🔍 Search movie by title..." required
                        style="padding: 10px; width: 80%; max-width: 300px; border-radius: 8px; border: none; font-size: 16px;">
                    <br>
                    <small style="color: #ccc; display: block; margin-top: 8px;">
                        Enter the movie's title (e.g., Inception, Titanic)
                    </small>
                    <button type="submit" style="margin-top: 12px; padding: 10px 20px; background-color: #ff4757; color: white;
                            border: none; border-radius: 5px; font-size: 16px;">
                        Search
                    </button>
                </form>
                {result_html}
            </body>
        </html>
    '''




# הרצת שרת Flask
if __name__ == '__main__':
    app.run(port=5000)

Flask app is running at: NgrokTunnel: "https://c7d8-34-125-133-223.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:02] "GET /static/qr_code.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:23] "GET /static/qr_code.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:24] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:25] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 11:53:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:16] "POST /predict HTTP/1

Form data received: ImmutableMultiDict([('title', 'Inception'), ('runtime', '120'), ('original_language', '0'), ('genres', 'drama'), ('spoken_languages', 'English'), ('year', '2007'), ('dayofweek', '1'), ('Directors', 'Dodi'), ('Stars', 'Frag'), ('Writers', 'Doli'), ('Rating_Count', '20000'), ('popularity', '89.7'), ('overview', 'The house is big'), ('plot', 'The house was biger and scary')])


INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:20] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:20] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:20] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:21] "GET /static/qr_code.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:41] "GET /search?query_title=Inception HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:49] "GET /static/qr_code.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:01:57] "GET /search?query_title=Scare HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:02:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:02:01] "GET /static/qr_code.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 12:02:08] "GET /se

Form data received: ImmutableMultiDict([('title', 'Inception'), ('runtime', '148'), ('original_language', '1'), ('genres', 'Action, Adventure, Thriller'), ('spoken_languages', 'English, Japanese, French'), ('year', '2010'), ('dayofweek', '1'), ('Directors', 'Christopher McQuarrie'), ('Stars', 'Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page'), ('Writers', 'Christopher McQuarrie'), ('Rating_Count', '-7000'), ('popularity', '87.7'), ('overview', 'A thief who enters the dreams of others to steal secrets from their subconscious is given the inverse task of planting an idea into the mind of a CEO.'), ('plot', ' As tensions rise across the galaxy, the crew must navigate betrayal, ancient secrets, and a ticking clock to complete their mission before universal war breaks out.')])
